# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686573


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:00,  2.15s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:48,  1.88s/it]

Rendering models:  20%|██        | 6/30 [00:06<00:32,  1.34s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:22,  1.03it/s]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  6.11it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  7.30it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.90it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  9.51it/s]

not-logged-in-68d3d82149e88f995533    0.480318
not-logged-in-1f0d49433a496578e774    0.001439
Staceyann116                          0.012060
kjkavene                              0.000189
shocko61                              0.000169
not-logged-in-3e831daf31432a82f496    0.039379
jmfranci                              0.000997
tosnyder                              0.000564
pollux2728                            0.006794
not-logged-in-ab5f58ee1e61681e9014    0.000493
ericmalani                            0.000714
not-logged-in-7e969a324309f376dde8    0.000861
orzechowskijacek                      0.003245
UnbiasedBrigade                       0.000967
gabrielweiss                          0.008419
zzw                                   0.003684
RaulE                                 0.000450
stansall                              0.000142
rhondacard                            0.000892
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000444
Lavadude                              0.013395
shoheitsuoyam

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())